In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '1.1.1'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
    return A, B, L

In [2]:
A, B, L = получить_матрицы_варианта(8)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)

In [3]:
def матрица_управляемости(A, B, L):
    матрица = sympy.eye(3)
    матрица[:, 0] = B 
    матрица[:, 1] = A*B
    матрица[:, 2] = A**2*B
    return матрица

In [4]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[ 8, -41,    3],
[-1,  36, -167],
[-3,  11,   36]])

In [5]:
def определитель_матрицы_управляемости(S):
    определитель = S.det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [6]:
print(определитель_матрицы_управляемости(S))

3338


In [7]:
def коэффициенты_характеристического_полином_разомкнутой_системы(A):
    l = sympy.symbols('lambda')
    I = sympy.eye(3)
    detChPolyOpen = (l*I - A).det()
    c0 = detChPolyOpen.coeff(l, 0)
    c1 = detChPolyOpen.coeff(l, 1)
    c2 = detChPolyOpen.coeff(l, 2)
    ChPolyOpen = sympy.Matrix([
        c0,
        c1,
        c2
        ])
    return ChPolyOpen

In [8]:
коэффициенты_характеристического_полином_разомкнутой_системы(A)

Matrix([
[152],
[ 59],
[ 11]])

In [9]:
def коэффициенты_характеристического_полином_замкнутой_системы(L):
    c0 = L[0]*L[1]*L[2]
    c1 = L[0]*L[1] + L[1]*L[2] + L[0]*L[2]
    c2 = L[0]+L[1]+L[2]

    ChPolyOpen = sympy.Matrix([
        -c0,
        c1,
        -c2
        ])
    return ChPolyOpen

In [10]:
коэффициенты_характеристического_полином_замкнутой_системы(L)

Matrix([
[15],
[23],
[ 9]])

In [34]:
def матрица_D_обратная(A, B, L):
    c0, c1, c2 = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    k0, k1, k2 = коэффициенты_характеристического_полином_замкнутой_системы(L)
    d3 = B
    d2 = A*d3 + c2*B
    d1 = A*d2 + c1*B
    res = sympy.eye(3)
    res[:, 0] = d1
    res[:, 1] = d2
    res[:, 2] = d3
    return res.inv().n()

In [35]:
матрица_D_обратная(A, B, L)

Matrix([
[0.0290593169562612,  0.0104853205512283, 0.0739964050329539],
[-0.158777711204314, -0.0263630916716597, -0.414619532654284],
[ 0.970641102456561,   0.123427201917316,   2.21390053924506]])

In [40]:
def вектор_G(A, L):
    return (коэффициенты_характеристического_полином_разомкнутой_системы(A) -\
        коэффициенты_характеристического_полином_замкнутой_системы(L)).T

In [41]:
вектор_G(A, L)

Matrix([[137, 36, 2]])

In [42]:
def вектор_p(A, B, L):
    G = вектор_G(A, L)
    D_обр = матрица_D_обратная(A, B, L)
    return G*D_обр

In [43]:
res = вектор_p(A, B, L)
res

Matrix([[0.206411024565608, 0.734272019173158, -0.36099460754943]])

In [44]:
toCopyString(res)

'matrix([0.206411024565608,0.734272019173158,-0.360994607549430])'

In [15]:
for var_number in range(1, 31):
    A, B, L = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)

    S                        = матрица_управляемости(A, B, L)
    определитель             = определитель_матрицы_управляемости(S)
    коэффициенты_разомкнутой = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    коэффициенты_замкнутой   = коэффициенты_характеристического_полином_замкнутой_системы(L)
    D_обртаная               = матрица_D_обратная(A, B, L)
    G                        = вектор_G(A, L)
    p                        = вектор_p(A, B, L)
    
    with open(f'./results/{var_number}.txt', 'w') as file:
        file.write(toCopyString(S) + '\n')
        file.write(str(определитель) + '\n')
        file.write(toCopyString(коэффициенты_разомкнутой) + '\n')
        file.write(toCopyString(коэффициенты_замкнутой) + '\n')
        file.write(toCopyString(D_обртаная) + '\n')
        file.write(toCopyString(G) + '\n')
        file.write(toCopyString(p) + '\n')
        